In [50]:
from urllib.request import Request, urlopen
import pandas as pd
from bs4 import BeautifulSoup
from argparse import ArgumentParser
import numpy as np

In [56]:
# Query the hockey-reference website for data and transform html into a readable table
html1 = urllib.request.urlopen("https://www.hockey-reference.com/leagues/NHL_2024_skaters.html#stats::points").read()
soup1 = BeautifulSoup(html1,"html.parser")

table1 = soup1.find_all('table')[0]
table_body1 = table1.find('tbody')
rows1 = table_body1.find_all('tr')

column_headers = ['Rk','Player','Age','Team','Pos','GP','G','A','PTS','+/-','PIM','EVG','PPG','SHG','GWG','EV','PP',
                  'SH','SOG','SPCT','TSA','TOI','ATOI','FOW','FOL','FO%','BL','HIT','TAKE','GIVE','Awards']

full_data = [[td1.getText() for td1 in rows1[i].find_all(['th','td'])]
            for i in range(len(rows1))]

In [58]:
df = pd.DataFrame(full_data, columns=column_headers)
print(df)

        Rk            Player Age Team Pos  GP   G    A  PTS +/-  ...      TOI  \
0        1   Nikita Kucherov  30  TBL  RW  81  44  100  144   8  ...  1755:10   
1        2  Nathan MacKinnon  28  COL   C  82  51   89  140  35  ...  1870:46   
2        3    Connor McDavid  27  EDM   C  76  32  100  132  35  ...  1623:18   
3        4    Artemi Panarin  32  NYR  LW  82  49   71  120  18  ...  1649:45   
4        5    David Pastrňák  27  BOS  RW  82  47   63  110  21  ...  1634:14   
...    ...               ...  ..  ...  ..  ..  ..  ...  ...  ..  ...      ...   
1164  1165     Vincent Iorio  21  WSH   D   6   0    0    0   1  ...    58:57   
1165  1166  Charlie Lindgren  30  WSH   G  50   0    0    0   0  ...  2852:23   
1166  1167    Hunter Shepard  28  WSH   G   4   0    0    0   0  ...   244:50   
1167  1168       Joe Snively  28  WSH   F   3   0    0    0  -1  ...    25:02   
1168          League Average                                     ...            

       ATOI  FOW  FOL   FO%

And there's our dataset! Looks like a few goalies slipped in, so let's clean those up.